# 1.  Discretization of gene expression values
Transform gene expression values into high (expressed / up-regulated) or low (not-expressed / down-regulated) gene expression binary equivalents

## 1.1. The expression values of a gene over the total number of input samples are sorted in descending order;

#### 1.2. The midpoints between each two consecutive values are calculated;

In [ ]:
def midpoint(num1,num2):
    return (num1+num2)/2

def sample_midpoints(sample):
    midpoints_dict={} # Key corresponds to the position of the first gene. The second gene is in the next position (i+1).
    for i in range(sample.shape[0]-1):
        midpoints_dict.update({i:midpoint(sample[i],sample[i+1])})
    return midpoints_dict

def get_midpoints(df):
    midpoints_dict={} # Key corresponds to the position of the first gene. The second gene is in the next position (i+1).
    for sample in range(df.shape[0]):
        midpoints_dict.update({sample:sample_midpoints(df.iloc[sample])})
    return midpoints_dict

#### 1.3. For each midpoint, μi, the Information Gain (IG) of the system is computed. Let IG(S,μi) to denote the IG of the system for midpoint μi.

In [ ]:
#classes=sorted(set(gse2034_df.index)) # Τhe classes to which a sample may belong
#samples=gse2034_df.index # the samples class

# Calculate the proportion of samples in S that belong in Class C
def P(C,S):
    return list(S).count(C)/len(S)

def E(S,m=1):
    # m not given: calculate the entropy of the system taking into account the prior assignment of sample cases into phenotype classes
    # m given: calculate the respective entropy of the system taking into account its division into subgroups around midpoint μi
    tmp=0
    for c in classes:
        # P(c,S) must be greater than zero
        tmp+=P(c,S)*math.log(P(c,S))/m
    return -(tmp)

#entropy=E(samples)

In [ ]:
# Calculate the Information Gain (IG) of the system
def IG(S,m):
    return E(S)-E(S,m)

#information_gain=[]
#for m in list(midpoints_dict.values()):
#    information_gain.append(IG(samples,m))
#print('Information Gain: '+str(information_gain))

In [ ]:
# The midpoint with the highest information gain is selected as the discretization point
#max_value = max(information_gain)
#max_mid_pos = information_gain.index(max(information_gain))
#dis_point=midpoints_dict.get(max_mid_pos)
#print('Discretization point: %.3f' %dis_point)

In [ ]:
#def sample_discretization(df):
    